#  Generating slip on fault planes from the European Fault-Source Model 2020 (EFSM20) database
This notebook will show you how to:
- import a mesh file from the [EFSM20 database](https://seismofaults.eu/efsm20)
- re-mesh the fault plane at a higher resolution  
- generate input files for _k223d_ (i.e. set nucleation location, rupture velocity, surface rupture)
- running _k223d_
- image rupture front and slip distribution 
- create an output file that can be read in QGis 

In [4]:
import numpy as np
import pandas as pd

import sys

# Mesh wrangling 
import gmsh
import meshio

# for running k223d
import subprocess

# track time of process
import time

# read in EFSM data set
import geopandas as gpd
import json
from geojson import Feature, FeatureCollection, Polygon, Point

# convert from Geographical coordinates 
import pyproj

#plotting mesh 
import plotly.graph_objs as go
import plotly.express as px


## Set Parameters and subroutines used in notebook

In [5]:
kilo = 1000.

### Custom Python modules

In [6]:
folder_path = '../local_py_scripts'
sys.path.append(folder_path)
from plotting import *
from notebook_utils import *
from fault_scaling import *

## Import EFSM fault from database 
A full description of the database and how to reference it can be found at: https://seismofaults.eu/efsm20 


In [7]:
gdf = gpd.read_file('ITCF02R.json')


In [8]:
# first extract the nodes and array linking nodes to cells entitled 'cells'
nodes, cells = extract_fault_data(gdf)
ncells = len(cells)
nnodes = len(nodes)
print("No of Nodes :   ",nnodes)
print("No of cells:  ",ncells)


No of Nodes :    36
No of cells:   49


## Earthquake of interest

We will take the $M_w 6.5$   Earthquake that occured in Norcia on 30/10/2016

Details for the event can be found at : https://terremoti.ingv.it/en/event/8863681


In [9]:
quake_lat = 42.83
quake_long = 13.109
quake_z = -10*kilo
quake_name = ['Norcia']
Mw = 6.5

In [ ]:
scale = kilo*10
points = np.transpose([nodes[:,0], nodes[:,1],nodes[:,2]/scale]) 
plot_mesh_quake(cells,points,quake_lat,quake_long,quake_z/scale)


## Convert from Lat/Lon to UTM 

In [11]:
utm_zone = str(int(np.floor((np.mean(nodes[:,1])  + 180) / 6 ) % 60) + 1)
if len(utm_zone) == 1:
    utm_zone = '0'+utm_zone
print('UTM Zone:     ',utm_zone)


UTM Zone:      33


In [12]:
# Parameters use in the script
# the first two relate to the converting lat/lon to utm and a geodesic distance
P = pyproj.Proj(proj='utm', zone=utm_zone, ellps='WGS84', preserve_units=True)    # took utm 

In [13]:
# convert fault coordinates to UTM coordinates 
# NOTE: all coorindates are set relative to the first nodal point in the mesh
#       is this useful? there could be difficulty converting the mesh back to 
#       geographical coordinates afterwards
UTM_x, UTM_y = P(nodes[:,1], nodes[:,0])
ref_x = 0. # UTM_x[0]
ref_y = 0. # UTM_y[0]
UTM_x = np.array(UTM_x)#-ref_x
UTM_y = np.array(UTM_y)#-ref_y

In [14]:
# convert fault epicentre to UTM coordinates and use same reference for origin
quake_x, quake_y = P(quake_long,quake_lat)
#quake_x = quake_x-ref_x
#quake_y = quake_y-ref_y

In [ ]:
points = np.transpose([UTM_x,UTM_y,nodes[:,2]]) 
plot_mesh_quake(cells,points,quake_x,quake_y,quake_z)

In [16]:
# If you just want to plot the mesh without the earthquake
# plot_mesh(cells,UTM_x,UTM_y,nodes[:,2])

Get an idea of area of cells on the mesh by calculating it's area

In [17]:
tri_area = tri_area_3d(points[cells])
ave_cell_area = np.mean(tri_area)
total_area = np.sum(tri_area)
edge_dist = tri_dist(points[cells])
ave_dist = np.mean(edge_dist)/kilo


max_mag = area2mag(total_area,'Strasser')
print("the largest earthquake possible on fault (Mw):   ",max_mag)
print("total number of cells:   ",ncells)
print("mean distance along side of cell is:   ",ave_dist,'km')

the largest earthquake possible on fault (Mw):    6.769684239799037
total number of cells:    49
mean distance along side of cell is:    5.2874953587633815 km


In [18]:
print('assume right angle (with two sides the average).....',1/2*ave_dist**2)

print('assume equilateral triangle.....',np.sqrt(3)/4*ave_dist**2)


assume right angle (with two sides the average)..... 13.97880358447215
assume equilateral triangle..... 12.105999018665852


How does this compare with the earthquake size we want to reproduce?
- the mag2area assumes that the magnitude is given as a magnitude and not a moment

In [19]:
# Mw = 6.0
target_area = mag2area(Mw,"Strasser")
print('Area of earthquake:',target_area/kilo**2,'km^2')
print("total size of fault surface:   ",np.sum(tri_area)/kilo**2,'km^2')
print("mean cell are:   ",ave_cell_area/kilo**2,'km^2')

print('No. of cells for rupture:',  int(target_area/ave_cell_area))

Area of earthquake: 515.2286445817562 km^2
total size of fault surface:    565.694353368649 km^2
mean cell are:    11.544782721809163 km^2
No. of cells for rupture: 44


In [20]:
target_no_cells = 2000
target_cell_area = (target_area/target_no_cells)
print('Aim to have cells with the following average area of.....:',target_cell_area/kilo**2,'km^2')
print('The current average area is ....:',ave_cell_area/kilo**2,'km^2')

Aim to have cells with the following average area of.....: 0.25761432229087805 km^2
The current average area is ....: 11.544782721809163 km^2


Assuming that the triangles are equilateral, we rearrange the area for triangle: 
$$ A = \frac{\sqrt{3}}{4} l^2$$
to calculate the length ,$l$, of one edge

In [21]:
target_edge = np.sqrt(target_cell_area*4./np.sqrt(3))   # assume triangles are 
print('This is the target average length of a cell edge.....',int(target_edge),'m')

This is the target average length of a cell edge..... 771 m


## Refine mesh in Gmsh

In [22]:
# remeshing fault, output is saved to file 'refined_mesh.vtk' viewable in paraview and 'refined_mesh.msh' which can be read by gmsh 
remesh_fault(points,cells,target_edge,'refined_mesh')

Info    : Clearing all models and views...
Info    : Done clearing all models and views
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 10%] Meshing curve 3 (Line)
Info    : [ 10%] Meshing curve 4 (Line)
Info    : [ 10%] Meshing curve 5 (Line)
Info    : [ 10%] Meshing curve 6 (Line)
Info    : [ 10%] Meshing curve 7 (Line)
Info    : [ 10%] Meshing curve 8 (Line)
Info    : [ 10%] Meshing curve 9 (Line)
Info    : [ 10%] Meshing curve 10 (Line)
Info    : [ 10%] Meshing curve 11 (Line)
Info    : [ 10%] Meshing curve 12 (Line)
Info    : [ 10%] Meshing curve 13 (Line)
Info    : [ 10%] Meshing curve 14 (Line)
Info    : [ 10%] Meshing curve 15 (Line)
Info    : [ 20%] Meshing curve 16 (Line)
Info    : [ 20%] Meshing curve 17 (Line)
Info    : [ 20%] Meshing curve 18 (Line)
Info    : [ 20%] Meshing curve 19 (Line)
Info    : [ 20%] Meshing curve 20 (Line)
Info    : [ 20%] Meshing curve 21 (Line)
Info    : [ 20%] Meshing curve 22 (L

## Extract mesh and check new cell size against target

In [23]:
new_x,new_y,new_z,new_cells = extract_mesh('refined_mesh.msh')

n_ncells = len(new_cells)
cell_type = np.zeros(n_ncells,dtype=int)
cell_vel = np.zeros(n_ncells,dtype=float)

Info    : Reading 'refined_mesh.msh'...
Info    : 232 entities
Info    : 1386 nodes
Info    : 3726 elements
Info    : Done reading 'refined_mesh.msh'
number of nodes      1386
number of elements     2621


In [24]:
# calculate new average area of a cell 
new_points = np.transpose([new_x,new_y,new_z])
print("No of nodes in new mesh:      ",len(new_points))

tri_area = tri_area_3d(new_points[new_cells])
print("mean cell are:   ",np.mean(tri_area)/kilo**2,'km^2')
print('No. of cells for rupture:',  int(target_area/np.mean(tri_area)))

No of nodes in new mesh:       1386
mean cell are:    0.21583149689761477 km^2
No. of cells for rupture: 2387


In [ ]:
# plot_mesh(new_cells,new_points) # if you wish to just see the new mesh
compare_meshes(cells,points,new_cells,new_points)


## Add rupture velocity to mesh
Here we set a depth dependent rupture velocity gradient

In [26]:
rupt_vel  = 2000.  # Set the initial velocity at the surface [m/s]
rupt_grad = 0.25   # Set the depth dependent velocity gradient [m/s per m] 

In [27]:
# set velocity for each cell
cell_vel = np.zeros(n_ncells)
for i in range(n_ncells):
    c_id = new_cells[i]
    # cell_vel[i] = rupt_vel    

    # use the mean depth of each cell
    mean_depth = np.abs(np.mean(new_z[c_id]))   
    cell_vel[i] = rupt_vel  + rupt_grad*(mean_depth)


In [ ]:
plot_mesh_cell(new_cells,new_points,cell_vel,'velocity')

### Nucleation location of earthquake  
To decide where to pick for the nucleation for the earthquake, we use the location given earlier from the INGV catalogue. It is also possible to decide your own location. To do this change the quake_x and quake_y values. Always check that the _assign_nucleation_location_ does not give an error due to the quake_x and quake_y values being outside of the mesh area. 


In [29]:
# plot_mesh(new_cells,new_points)
# quake_x =-238.3*kilo
# quake_y= -96*kilo
# only using x,y coordinates for defining earthquake initiation 
# assuming rupture velocity is constant within cells 
itime = assign_nucleation_location(quake_x,quake_y,new_x,new_y,new_z,new_cells,cell_vel)

Source is in cell 894
Initial travel time on starting cell edges: [0.04685739 0.09773285 0.08865303]


In [ ]:
plot_mesh_node(new_cells,new_points,itime,'time')

## Asign surface rupture 
First we look at the depth range on the fault from this choose if we wish to set the upper nodes as a 'surface' to cause large slip at the surface

In [31]:
print('fault depth range (m):    ',np.min(new_z),'to ' ,np.max(new_z))

fault depth range (m):     -10000.0 to  -1500.0


In [32]:
# set surface rupture depth 
surface_depth = -1500.0 
new_nnodes = len(new_z)
surface_nodes =  np.zeros(new_nnodes,dtype=int)
for i in range(new_nnodes):
    if new_z[i] >= surface_depth:
        surface_nodes[i] = 1
print('number of surface nodes.....',np.sum(surface_nodes))

number of surface nodes..... 64


In [ ]:
plot_highlight_nodes(new_cells,new_points,surface_nodes,'surface nodes')


# Output mesh for k223d
Save all fields to a vtk file that will be read by _k223d_

In [34]:
mesh_file = 'input.vtk'
write_vtk(mesh_file,new_points,new_cells,cell_vel,itime,surface_nodes)

# Prepare input file 

In [37]:
# Make changes to inputfile 
# specify : earthquake size,name of the file containing the mesh, name of the output file  
file_path = 'input_file'
updates = {
    'magnitude':Mw,
    'mesh_file':mesh_file,
    'out_file':'prova2b',
    'defined_area':'.true.' 
}

update_inputfile(file_path, updates)

## Run k223d

In [38]:
# call k223d and print elapsed time
start=time.time()
subprocess.call(["../../code/k223d.x"])
end=time.time()
print('time for comuptation....',end-start)

 In single use mode
 enter read_vtk_mesh
 No. of nodes....        1386
 no. of cells........         2621
 number of nucleation nodes found...           3
         270         829         842
 nucleation cell id:              895
 fault  surface found
 no. of surface nodes:               64
 exit read_vtk_mesh
 starting allocation
 create FVsNodes
 start EToE
 starting trilateration.....
 exiting based on fast protocol ....
time for comuptation.... 2.8402481079101562


## Read in Results and plot

In [39]:
res = meshio.read("prova2b.vtk")
tri = res.get_cells_type("triangle")

a = np.array(res.cell_data['vel'])
res_vel = a.flatten('F')

a = np.array(res.cell_data['slip'])
res_slip = a.flatten('F')

a = np.array(res.point_data['Rupt_time'])
res_time = a.flatten('F')

In [ ]:
plot_mesh_node(tri,res.points,res_time,'Rupture Time (s)')

In [ ]:
plot_mesh_cell(tri,res.points,res_slip,'slip (m)')

## Convert to Geographical Coordinates 

In [ ]:
slip_lon, slip_lat = P(res.points[:,0], res.points[:,1], inverse=True)
nodes = np.transpose([slip_lon, slip_lat,res.points[:,2]]) 

# Output for QGIS
Output a geojson file with geographical coordinates so the slip and rupture time can be viewed in QGIS. 

In [40]:
write2gis(nodes,tri,res_slip,res_time,"quake_mesh.geojson")

GeoJSON file created with name  quake_mesh.geojson
